In [4]:
# 카드정보불러오기기
import pandas as pd
df = pd.read_csv("data/bus.csv")
#정류장 정보
df_bus_stop = pd.read_csv("data/a.csv", encoding='euc-kr')

In [5]:
new_df = df[df['card_rout_cd'] == 60600089]
new_df['승차정류장ID'] = new_df['card_brd_sttn_id']
new_df['하차정류장ID'] = new_df['card_alit_sttn_id']

/tmp/ipykernel_4167018/1303892172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['승차정류장ID'] = new_df['card_brd_sttn_id']
/tmp/ipykernel_4167018/1303892172.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['하차정류장ID'] = new_df['card_alit_sttn_id']


In [6]:
# 특정 노선을 지정하여 할 경우
id_to_name = df_bus_stop.set_index('정류장ID')['버스정류장명'].to_dict()

# df_trx에 정류장명 컬럼 추가
new_df['승차버스정류장명'] = new_df['승차정류장ID'].map(id_to_name)

# df_trx에 정류장명 컬럼 추가
new_df['하차버스정류장명'] = new_df['하차정류장ID'].map(id_to_name)

/tmp/ipykernel_4167018/728916807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['승차버스정류장명'] = new_df['승차정류장ID'].map(id_to_name)
/tmp/ipykernel_4167018/728916807.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['하차버스정류장명'] = new_df['하차정류장ID'].map(id_to_name)


In [8]:
new_df = new_df.dropna(subset=['하차버스정류장명'])
dropnan_df = new_df.dropna(subset=['승차버스정류장명'])
test = dropnan_df

In [9]:
# 기존 시간 초로 변환 함수
def to_seconds(time_val):
    if pd.isnull(time_val): return None
    
    # 숫자를 문자열로 변환하고 길이에 따라 처리
    time_val = str(int(time_val)).zfill(6)  # 최소 6자리로 채움, 예: 011101 -> 00시11분01초
    
    if len(time_val) == 4:  # 4자리 (HHMM)
        hh, mm = int(time_val[:2]), int(time_val[2:])
        ss = 0  # 초는 0으로 설정
    elif len(time_val) == 6:  # 6자리 (HHMMSS)
        hh, mm, ss = int(time_val[:2]), int(time_val[2:4]), int(time_val[4:])
    
    return hh * 3600 + mm * 60 + ss

# 시간 계산
test['탑승초'] = test['card_brd_tm'].apply(to_seconds)
test['하차초'] = test['alit_deal_tm'].apply(to_seconds)

# 날짜 차이 계산 (같은 날이면 0, 다음 날이면 +1일 고려)
test['탑승일'] = pd.to_datetime(test['brd_deal_dt'], format='%Y%m%d', errors='coerce')
test['하차일'] = pd.to_datetime(test['alit_deal_dt'], format='%Y%m%d', errors='coerce')

# 전체 초 계산
test['탑승총초'] = test['탑승일'].astype('int64') // 10**9 + test['탑승초']
test['하차총초'] = test['하차일'].astype('int64') // 10**9 + test['하차초']

# 소요 시간 계산
test['소요시간(초)'] = test['하차총초'] - test['탑승총초']

# 음수 제거
test = test[test['소요시간(초)'] >= 0]

# 이름 기준으로 평균 소요 시간 계산

route = [
    '반석역', '세종고속시외버스터미널(지상)', '세종고속시외버스터미널(지하)', '한솔동',
    '새롬동,나성동(청소년센터)', '다정동', '정부세종청사남측', '정부세종청사북측', '도담동',
    '세종충남대학교병원', '해밀동', '한별동', '누리동', '오송역'
]

#route = test['승차버스정류장명'].unique()
avg_times = []

for i in range(len(route) - 1):
    출발 = route[i]
    도착 = route[i + 1]
    
    df_segment = test[
        (test['승차버스정류장명'] == 출발) &
        (test['하차버스정류장명'] == 도착)
    ]
    
    if not df_segment.empty:
        avg_time_min = df_segment['소요시간(초)'].mean() / 60
        avg_times.append({
            '출발': 출발,
            '도착': 도착,
            '평균소요시간(분)': round(avg_time_min, 2)
        })

# 결과 출력
result_df = pd.DataFrame(avg_times)
result_df

,출발,도착,평균소요시간(분)
0,반석역,세종고속시외버스터미널(지상),12.48
1,세종고속시외버스터미널(지상),세종고속시외버스터미널(지하),1.86
2,세종고속시외버스터미널(지하),한솔동,3.72
3,한솔동,"새롬동,나성동(청소년센터)",1.64
4,"새롬동,나성동(청소년센터)",다정동,1.78
5,다정동,정부세종청사남측,1.61
6,정부세종청사남측,정부세종청사북측,1.74
7,정부세종청사북측,도담동,1.96
8,도담동,세종충남대학교병원,1.83
9,세종충남대학교병원,해밀동,1.29
